<a href="https://colab.research.google.com/github/CC4mpbe11/MushroomClassification/blob/main/MushroomClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow-gpu

In [ ]:
pip install -q sklearn

In [4]:
pip install -q keras

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
from six.moves import urllib
from IPython.display import clear_output
from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from pathlib import Path  

import io
import tensorflow as tf
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100
import tensorflow.compat.v2.feature_column as fc

In [ ]:
mushroom_data = df[['edibility', # x
          'cap-diameter', #
          'cap-shape', #
          'cap-surface', # new category
          'cap-color', #
          'does-bruise-or-bleed', #
          'gill-attachment', # new category
          'gill-color', #
          'stem-height', #
          'stem-width', #
          'stem-color', #
          'has-ring', #
          'ring-type', # new category
          'habitat', #
          'season']]

In [ ]:
mushroom_data["cap-surface"].fillna("n", inplace = True)
mushroom_data["gill-attachment"].fillna("n", inplace = True)
mushroom_data["ring-type"].fillna("n", inplace = True)

In [ ]:
y = mushroom_data['edibility']

In [ ]:
x.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,15.26,x,g,o,0,e,w,16.95,17.09,w,1,g,d,w
1,16.60,x,g,o,0,e,w,17.99,18.19,w,1,g,d,u
2,14.07,x,g,o,0,e,w,17.80,17.74,w,1,g,d,w
3,14.17,f,h,e,0,e,w,15.77,15.98,w,1,p,d,w
4,14.64,x,h,o,0,e,w,16.53,17.20,w,1,p,d,w


In [ ]:
y.head()

In [ ]:
x = mushroom_data[['cap-diameter',
          'cap-shape',
          'cap-surface',
          'cap-color',
          'does-bruise-or-bleed',
          'gill-attachment',
          'gill-color',
          'stem-height',
          'stem-width',
          'stem-color',
          'has-ring',
          'ring-type',
          'habitat',
          'season']]

In [ ]:
x["cap-surface"].fillna("n", inplace = True)
x["gill-attachment"].fillna("n", inplace = True)
x["ring-type"].fillna("n", inplace = True)

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load

mushroom_data_path = Path('/content/drive/MyDrive/Datasets/Mushroom Data/mushroom_data.csv')
mushroom_data = pd.read_csv(mushroom_data_path) 

In [ ]:
ypath = Path('/content/drive/MyDrive/Datasets/Mushroom Data/y.csv')
y = pd.read_csv(ypath)

In [ ]:
xpath = Path('/content/drive/MyDrive/Datasets/Mushroom Data/x.csv')
x = pd.read_csv(xpath)

In [ ]:
# Save

mushroom_data_path = Path('/content/drive/MyDrive/Datasets/Mushroom Data/mushroom_data.csv')
mushroom_data_path.parent.mkdir(parents=True, exist_ok=True)
mushroom_data.to_csv(mushroom_data_path, index=False) 

In [ ]:
ypath = Path('/content/drive/MyDrive/Datasets/Mushroom Data/y.csv')  
ypath.parent.mkdir(parents=True, exist_ok=True)  
y.to_csv(ypath, index=False) 

In [ ]:
xpath = Path('/content/drive/MyDrive/Datasets/Mushroom Data/x.csv')  
xpath.parent.mkdir(parents=True, exist_ok=True)  
x.to_csv(xpath, index=False) 

In [ ]:
# Replace edibility with 1 for edible, and 0 for poisonous

mushroom_data['edibility'] = mushroom_data['edibility'].replace("p", 0)
mushroom_data['edibility'] = mushroom_data['edibility'].replace("e", 1)

mushroom_data['does-bruise-or-bleed'] = mushroom_data['does-bruise-or-bleed'].replace("t", 1)
mushroom_data['does-bruise-or-bleed'] = mushroom_data['does-bruise-or-bleed'].replace("f", 0)

mushroom_data['has-ring'] = mushroom_data['has-ring'].replace("t", 1)
mushroom_data['has-ring'] = mushroom_data['has-ring'].replace("f", 0)

In [ ]:
y['edibility'] = y['edibility'].replace("p", 0)
y['edibility'] = y['edibility'].replace("e", 1)

In [ ]:
x['does-bruise-or-bleed'] = x['does-bruise-or-bleed'].replace("t", 1)
x['does-bruise-or-bleed'] = x['does-bruise-or-bleed'].replace("f", 0)

x['has-ring'] = x['has-ring'].replace("t", 1)
x['has-ring'] = x['has-ring'].replace("f", 0)

In [ ]:
y.dtype

dtype('int64')

In [ ]:
y_float = pd.to_numeric(y, downcast='float')

In [ ]:
ohe = OneHotEncoder(sparse = False)

In [ ]:
ohe.fit_transform(mushroom_data[['cap-shape']])

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [ ]:
cap_diam_col = mushroom_data['cap-diameter']
stem_width_col = mushroom_data['stem-width']
stem_height_col = mushroom_data['stem-height']

stem_width_col.shape

(61069,)

In [ ]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['cap-shape',
                       'cap-surface',
                       'cap-color',
                       'gill-attachment',
                       'gill-color',
                       'stem-color',
                       'ring-type',
                       'habitat',
                       'season']), 
                       remainder = 'passthrough')

In [ ]:
pd.get_dummies(x['cap-shape'], prefix='cap-shape')
#
#
#
#

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,cap-shape_s,cap-shape_x
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
61064,0,0,0,0,0,1,0
61065,0,0,1,0,0,0,0
61066,0,0,0,0,0,1,0
61067,0,0,1,0,0,0,0


In [ ]:
empty_list = []

for col_name in x.drop(['cap-diameter', 'stem-width', 'stem-height','does-bruise-or-bleed','has-ring'], axis=1).columns:
    empty_list.append(pd.get_dummies(x[col_name], prefix=col_name))
dummies_df = pd.concat(empty_list, axis=1)


In [ ]:
dummies_df.head().T

In [ ]:
dummies_df['cap-diameter'] = x['cap-diameter']
dummies_df['stem-width'] = x['stem-width']
dummies_df['stem-height'] = x['stem-height']
dummies_df['does-bruise-or-bleed'] = x['does-bruise-or-bleed']
dummies_df['has-ring'] = x['has-ring']

In [ ]:
dummies_df.head().T

In [ ]:
dummies_df.shape

(61069, 90)

In [ ]:
cap_shape_dict = {'b':'csh_bell'
, 'c':'csh_conical'
, 'x':'csh_convex'
, 'f':'csh_flat'}

x.replace({'cap-shape':cap_shape_dict}, inplace=True)

cap_surface_dict = {'i':'cs_fibrous'
, 'g':'cs_grooves'
, 'y':'cs_scaly'
, 's':'cs_smooth'
, 'h':'cs_shiny'
, '1':'cs_leathery'
, 'k':'cs_silky'
, 't':'cs_sticky'
, 'w':'cs_wrinkled'
, 'e':'cs_fleshy'}

x.replace({'cap-surface':cap_surface_dict}, inplace=True)

cap_color_dict = {'n':'cc_brown'
, 'b':'cc_buff'
, 'g':'cc_gray'
, 'r':'cc_green'
, 'p':'cc_pink'
, 'u':'cc_purple'
, 'e':'cc_red'
, 'w':'cc_white'
, 'y':'cc_yellow'
, 'l':'cc_blue'
, 'o':'cc_orange'
, 'k':'cc_black'}

x.replace({'cap-color':cap_color_dict}, inplace=True)

gill_attach_dict = {'a':'ga_adnate'
, 'x':'ga_adnexed'
, 'd':'ga_decurrent'
, 'e':'ga_free'
, 's':'ga_sinuate'
, 'p':'ga_pores'
, 'f':'ga_none'
, 'n':'ga_noattachment'}

x.replace({'gill-attachment':gill_attach_dict}, inplace=True)

gill_color_dict = {'n':'gc_brown'
, 'b':'gc_buff'
, 'g':'gc_gray'
, 'r':'gc_green'
, 'p':'gc_pink'
, 'u':'gc_purple'
, 'e':'gc_red'
, 'w':'gc_white'
, 'y':'gc_yellow'
, 'l':'gc_blue'
, 'o':'gc_orange'
, 'k':'gc_black'
, 'f':'gc_none'}

x.replace({'gill-color':gill_color_dict}, inplace=True)

stem_color_dict = {'n':'sc_brown'
, 'b':'sc_buff'
, 'g':'sc_gray'
, 'r':'sc_green'
, 'p':'sc_pink'
, 'u':'sc_purple'
, 'e':'sc_red'
, 'w':'sc_white'
, 'y':'sc_yellow'
, 'l':'sc_blue'
, 'o':'sc_orange'
, 'k':'sc_black'
, 'f':'sc_none'}

x.replace({'stem-color':stem_color_dict}, inplace=True)

ring_type_dict = {'c':'rt_cobwebby'
, 'e':'rt_evanescent'
, 'r':'rt_flaring'
, 'g':'rt_grooved'
, 'l':'rt_large'
, 'p':'rt_pendant'
, 's':'rt_sheathing'
, 'z':'rt_zone'
, 'y':'rt_scaly'
, 'm':'rt_movable'
, 'f':'rt_none'}

x.replace({'ring-type':ring_type_dict}, inplace=True)

habitat_dict = {'g':'rtt_grasses'
, 'l':'rtt_leaves'
, 'm':'rtt_meadows'
, 'p':'rtt_paths'
, 'h':'rtt_heaths'
, 'u':'rtt_urban'
, 'w':'rtt_waste'
, 'd':'rtt_woods'}

x.replace({'habitat':habitat_dict}, inplace=True)

season_dict = {'s':'s_spring'
, 'u':'s_summer'
, 'a':'s_autumn'
, 'w':'s_winter'}

x.replace({'season':season_dict}, inplace=True)

In [ ]:
x = mushroom_data.drop('edibility', axis='columns')

In [ ]:
logreg = LogisticRegression(solver='sag', max_iter=100)

In [ ]:
x_cap_diam = x.loc[:,['cap-diameter']]
x_stem_width = x.loc[:,['stem-width']]
x_stem_height = x.loc[:,['stem-height']]
#y = mushroom_data.edibility

In [ ]:
cross_val_score(logreg, x_cap_diam, y, cv=5, scoring='accuracy').mean()
#cross_val_score(logreg, x_stem_width, y, cv=5, scoring='accuracy').mean()
#cross_val_score(logreg, x_stem_height, y, cv=5, scoring='accuracy').mean()

In [ ]:
column_trans.fit_transform(x)

In [ ]:
logreg.solver

'sag'

In [ ]:
pipe = make_pipeline(column_trans, logreg)

In [ ]:
#cross_val_score(logreg, dummies_df, y, cv=5, scoring='accuracy').mean()
#cross_val_score(logreg, dummies_df, y, cv=5, scoring='accuracy').mean()
cross_val_score(pipe, dummies_df, y, cv=5, scoring='accuracy').mean()

In [ ]:
y_train, y_test, x_train, x_test = train_test_split(y, x, test_size=0.1)
#y_train
#y_test
#x_train
#x_test

In [ ]:
CATEGORICAL_COLUMNS = ['cap-shape','cap-surface', 'cap-color', 'gill-attachment',
                       'gill-color', 'stem-color', 'ring-type', 'habitat', 'season']
NUMERIC_COLUMNS = ['cap-diameter', 'stem-height', 'stem-width', 'does-bruise-or-bleed', 'has-ring']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = x_train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [ ]:
x_train.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
36298,5.97,x,h,n,f,x,w,5.94,6.11,n,t,r,d,w
50664,3.65,x,d,n,f,p,n,4.11,9.14,y,f,f,d,a
30705,10.92,s,t,p,t,d,o,6.31,18.15,w,f,f,d,u
7515,4.03,f,l,y,f,x,y,5.61,4.78,y,f,f,l,u
56219,8.91,o,e,r,f,n,o,1.60,13.40,n,f,f,d,w


In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(x_train, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(x_test, y_test, num_epochs=1, shuffle=False)

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

In [ ]:
print(result['accuracy'])

0.7812346
